In [1]:
import numpy as np
import pandas as pd
import dtale #!pip install dtale
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import neighbors
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

hello world


In [2]:
df=pd.read_csv('data/japan_bank_500_conNulos.csv')
dtale.show(df)

2021-10-04 07:58:00,958 - INFO     - NumExpr defaulting to 8 threads.


In [3]:
df_copy = df #Genero una copia por las dudas
X = df_copy.drop(["y"], axis=1)
Y = df_copy["y"]

In [4]:
#divido en train y test 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 123)

In [5]:
#divido en train_validation y test_validation
X_val_train, X_val_test, Y_val_train, Y_val_test = train_test_split(X_train,Y_train,test_size=0.2,random_state=123)

### Limpieza de los Datos

In [7]:
imputador=SimpleImputer(strategy="median")

In [6]:
df_copy.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,y
0,b,29.83,NaN,NaN,g,NaN,v,NaN,f,f,0.0,f,NaN,216.0,0.0,-
1,NaN,36.75,125.00,y,p,NaN,v,NaN,f,f,0.0,t,g,232.0,NaN,+
2,NaN,24.92,NaN,u,g,ff,NaN,0.00,t,f,NaN,f,NaN,80.0,0.0,-
3,b,29.67,0.75,y,NaN,c,NaN,0.04,NaN,f,NaN,f,NaN,NaN,0.0,-
4,NaN,22.17,12125.00,NaN,g,NaN,v,3335.00,NaN,NaN,2.0,NaN,g,180.0,NaN,-


In [9]:
cols_num=X_val_train.select_dtypes(np.number).columns  
cols_cat=X_val_train.select_dtypes(include=['object']).columns  


t1_s=('imputador_num', SimpleImputer(strategy='median'), cols_num)  
t2_s=('imputador_cat', SimpleImputer(strategy='most_frequent'), cols_cat)  

transformador_columnas = ColumnTransformer(transformers=[t1_s,t2_s],remainder='passthrough',n_jobs=-1)  
transformador_columnas.fit(X_val_train)  
X_val_train_Transformado = transformador_columnas.transform(X_val_train) 

X_val_train_Transformado=pd.DataFrame(X_val_train_Transformado, columns=X_val_train.columns) 
X_val_train_Transformado

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
0,28.665,15.5,3.0,0.0,162.0,0.0,b,u,g,c,v,f,f,f,g
1,28.665,7.0,1085.0,0.0,320.0,6.0,b,u,g,c,v,f,f,f,g
2,24.58,12.5,875.0,0.0,162.0,6.0,b,u,g,c,v,t,f,f,g
3,18.83,4415.0,3.0,0.0,240.0,6.0,a,u,p,c,v,f,f,f,g
4,28.665,2.5,3.0,7.0,150.0,1270.0,b,u,g,c,v,t,t,f,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,13.75,7.0,1.75,0.0,162.0,1000.0,b,u,p,c,v,t,f,t,g
316,28.665,10.75,415.0,5.0,0.0,6.0,a,u,g,c,v,f,t,f,g
317,25.25,1.0,0.5,0.0,200.0,0.0,b,u,g,c,v,f,f,f,g
318,25.83,12835.0,0.5,0.0,0.0,6.0,b,u,g,c,v,f,f,f,g


In [10]:
X_val_test_Transformado = transformador_columnas.transform(X_val_test)
X_val_test_Transformado = pd.DataFrame(X_val_test_Transformado, columns=X_val_test.columns) 
#X_val_test_Transformado

In [11]:
# imputador kNN
t1=('imputador_num1', KNNImputer(n_neighbors=5), cols_num)  
t2=('imputador_cat1', SimpleImputer(strategy='most_frequent'), cols_cat) 

#Transformador de Columnas
transformador_columnas_knn = ColumnTransformer(transformers=[t1,t2],remainder='passthrough',n_jobs=-1)  

# fit en el Train
transformador_columnas_knn.fit(X_val_train)

# Lo aplico en el Train 
X_val_train_Transformado_knn = transformador_columnas_knn.transform(X_val_train) 
X_val_train_Transformado_knn = pd.DataFrame(X_val_train_Transformado_knn, columns=X_val_train.columns) 
#X_val_train_Transformado_knn

In [12]:
# Lo aplico en el Test
X_val_test_Transformado_knn = transformador_columnas_knn.transform(X_val_test) 
X_val_test_Transformado_knn = pd.DataFrame(X_val_test_Transformado_knn, columns=X_val_test.columns) 
#X_val_test_Transformado_knn

# dummy simple

# normalizador

In [13]:
X_all=X_val_train_Transformado.append(X_val_test_Transformado,ignore_index=True)
X_all

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
0,28.665,15.5,3.0,0.0,162.0,0.0,b,u,g,c,v,f,f,f,g
1,28.665,7.0,1085.0,0.0,320.0,6.0,b,u,g,c,v,f,f,f,g
2,24.58,12.5,875.0,0.0,162.0,6.0,b,u,g,c,v,t,f,f,g
3,18.83,4415.0,3.0,0.0,240.0,6.0,a,u,p,c,v,f,f,f,g
4,28.665,2.5,3.0,7.0,150.0,1270.0,b,u,g,c,v,t,t,f,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,22.67,7.0,1335.0,0.0,100.0,6.0,b,u,g,q,v,f,f,f,g
396,23.08,11.5,3.0,0.0,290.0,6.0,b,u,g,c,h,f,t,t,g
397,28.665,375.0,3.0,0.0,162.0,6.0,b,u,g,c,v,f,f,f,g
398,48.33,7.0,3.0,0.0,162.0,0.0,b,u,g,c,v,f,f,f,s


In [15]:
tdummy=("onehot",OneHotEncoder(sparse=False), ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15'])
t1_norm_simp=("normalizador",MinMaxScaler(feature_range=(0, 1)),['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])
transformador_columnas_dummy = ColumnTransformer(transformers=[tdummy,t1_norm_simp],remainder='passthrough')


Executing shutdown due to inactivity...


2021-10-04 08:58:20,400 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2021-10-04 08:58:32,735 - INFO     - Executing shutdown...


In [121]:
transformador_columnas_dummy.fit(X_all)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(sparse=False),
                                 ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13',
                                  'x14', 'x15']),
                                ('normalizador1', MinMaxScaler(),
                                 ['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])])

In [122]:
# Lo aplicamos al X_val_train
X_train_dummy=transformador_columnas_dummy.transform(X_val_train_Transformado)
X_train_dummy_Simple = X_train_dummy
#X_train_dummy_Simple

In [123]:
# Lo aplicamos al X_val_test
X_test_dummy=transformador_columnas_dummy.transform(X_val_test_Transformado)
X_test_dummy_Simple = X_test_dummy
#X_test_dummy_Simple

# Estanarizador

In [128]:
t1_norm_stand=("estanarizador",StandardScaler(),['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])
transformador_columnas_dummy_estand = ColumnTransformer(transformers=[tdummy,t1_norm_stand],remainder='passthrough')

In [129]:
transformador_columnas_dummy_estand.fit(X_all)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(sparse=False),
                                 ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13',
                                  'x14', 'x15']),
                                ('estanarizador', StandardScaler(),
                                 ['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])])

In [131]:
# Lo aplicamos al X_val_train
X_train_dummy_stand=transformador_columnas_dummy_estand.transform(X_val_train_Transformado)
X_train_dummy_Simple_stand = X_train_dummy_stand
#X_train_dummy_Simple_stand

In [132]:
# Lo aplicamos al X_val_test
X_test_dummy_stand=transformador_columnas_dummy_estand.transform(X_val_test_Transformado)
X_test_dummy_Simple_stand = X_test_dummy_stand
#X_test_dummy_Simple

# dummy kNN

# normalizador

In [117]:
X_all_knn=X_val_train_Transformado_knn.append(X_val_test_Transformado_knn,ignore_index=True)
#X_all_knn

In [135]:
tdummy_knn=("onehot",OneHotEncoder(sparse=False), ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15'])
t1_norm_knn=("normalizador_k",MinMaxScaler(feature_range=(0, 1)),['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])
transformador_columnas_dummy_knn = ColumnTransformer(transformers=[tdummy_knn,t1_norm_knn],remainder='passthrough')

In [136]:
transformador_columnas_dummy_knn.fit(X_all_knn)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(sparse=False),
                                 ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13',
                                  'x14', 'x15']),
                                ('normalizador_k', MinMaxScaler(),
                                 ['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])])

In [137]:
# Lo aplicamos al X_val_train
X_train_dummy_knn=transformador_columnas_dummy_knn.transform(X_val_train_Transformado_knn)
X_train_dummy_knn = X_train_dummy_knn
#X_train_dummy_knn

In [140]:
# Lo aplicamos al X_val_test
X_test_dummy_knn=transformador_columnas_dummy_knn.transform(X_val_test_Transformado_knn)
X_test_dummy_knn = X_test_dummy_knn
#X_test_dummy_knn

# Estanarizador

In [141]:
t1_norm_stand_knn=("estanarizador",StandardScaler(),['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])
transformador_columnas_dummy_estand_knn = ColumnTransformer(transformers=[tdummy,t1_norm_stand_knn],remainder='passthrough')

In [143]:
transformador_columnas_dummy_estand_knn.fit(X_all_knn)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(sparse=False),
                                 ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13',
                                  'x14', 'x15']),
                                ('estanarizador', StandardScaler(),
                                 ['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])])

In [147]:
# Lo aplicamos al X_val_train
X_train_dummy_stand_knn=transformador_columnas_dummy_estand_knn.transform(X_val_train_Transformado_knn)
#X_train_dummy_stand_knn 

In [149]:
# Lo aplicamos al X_val_test
X_test_dummy_stand_knn=transformador_columnas_dummy_estand_knn.transform(X_val_test_Transformado_knn)
#X_test_dummy_stand_knn

In [160]:
sim_norm=DecisionTreeClassifier(max_depth=4)
sim_norm.fit(X_train_dummy_Simple,Y_val_train) 
sim_pred_norm = sim_norm.predict(X_test_dummy_Simple) 

AC=accuracy_score(Y_val_test, sim_pred_norm)
F1=f1_score(Y_val_test,sim_pred_norm, average='macro')  
print("Evaluación modelo Árbol de Decisión dummy simple ( con normalizacion) ")
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo Árbol de Decisión dummy simple ( con normalizacion) 
Accuracy:  0.738
F1:  0.732


In [162]:
sim_stan=DecisionTreeClassifier(max_depth=4)
sim_stan.fit(X_train_dummy_Simple_stand,Y_val_train) 
sim_pred_stand = sim_stan.predict(X_test_dummy_Simple_stand) 

AC=accuracy_score(Y_val_test, sim_pred_stand)
F1=f1_score(Y_val_test,sim_pred_stand, average='macro')  
print("Evaluación modelo Árbol de Decisión dummy simple ( con Estanarizador) ")
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo Árbol de Decisión dummy simple ( con Estanarizador) 
Accuracy:  0.738
F1:  0.732


In [163]:
knn_norm=DecisionTreeClassifier(max_depth=4)
knn_norm.fit(X_train_dummy_knn,Y_val_train) 
knn_pred_norm = knn_norm.predict(X_test_dummy_knn) 

AC=accuracy_score(Y_val_test, knn_pred_norm)
F1=f1_score(Y_val_test,knn_pred_norm, average='macro')  
print("Evaluación modelo Árbol de Decisión kNN ( con normalizacion) ")
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo Árbol de Decisión kNN ( con normalizacion) 
Accuracy:  0.788
F1:  0.785


In [164]:
knn_stand=DecisionTreeClassifier(max_depth=4)
knn_stand.fit(X_train_dummy_stand_knn,Y_val_train) 
knn_pred_stand = knn_stand.predict(X_test_dummy_stand_knn) 

AC=accuracy_score(Y_val_test, knn_pred_stand)
F1=f1_score(Y_val_test,knn_pred_stand, average='macro')  
print("Evaluación modelo Árbol de Decisión kNN ( con Estanarizador) ")
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo Árbol de Decisión kNN ( con Estanarizador) 
Accuracy:  0.788
F1:  0.785


## Segunda Parte:  

Una vez decidido cómo imputará deberá elegir el modelo que mejor sirve para pronosticar los valores del target.   
- Tenga en cuenta que como hará selección de modelo e hiperparámetros deberá compararlos en el Validation.     
- Una vez elegido el mejor modelo, deberá indicar cuál es la Accuracy Esperada cuando el modelo se use con nuevas observaciones.   
 En la celda siguiente indique cuál fue:  
 
 - modelo elegido:  
 - hiperparámetros:  
 - Accuracy Esperada

# Pruebo modelo de arbol con sus mejores hiperparametros

In [166]:
Arb_pru=DecisionTreeClassifier(max_depth=2, random_state=123)
Arb_pru.fit(X_train_dummy_stand_knn,Y_val_train) 
Ar_pred = Arb_pru.predict(X_test_dummy_stand_knn)
AC=accuracy_score(Y_val_test, Ar_pred)
F1=f1_score(Y_val_test,Ar_pred, average='macro') 
    

print("Evaluación modelo Árbol de Decisión")
print('profundidad del arbol: 2')
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo Árbol de Decisión
profundidad del arbol: 2
Accuracy:  0.788
F1:  0.777


# Pruebo modelo kNN con sus mejores hiperparametros

In [169]:
knn_pru=neighbors.KNeighborsClassifier(n_neighbors=7,weights='uniform')
knn_pru.fit(X_train_dummy_stand_knn,Y_val_train) 
knn_pred = knn_pru.predict(X_test_dummy_stand_knn)
AC=accuracy_score(Y_val_test, Ar_pred)
F1=f1_score(Y_val_test,knn_pred, average='macro') 
    

print("Evaluación modelo kNN")
print('Vecinos: 7')
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo kNN
Vecinos: 7
Accuracy:  0.788
F1:  0.73


# Modelo train

In [201]:
cols_n=X_train.select_dtypes(np.number).columns  
cols_c=X_train.select_dtypes(include=['object']).columns 

In [202]:
t1_model=('imputador_n', KNNImputer(n_neighbors= 5),cols_n)
t2_model=('imputador_c',SimpleImputer(strategy='most_frequent'),cols_c)

transformador_columnas=ColumnTransformer(transformers=[t1_model,t2_model],remainder='passthrough')
transformador_columnas.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('imputador_n', KNNImputer(),
                                 Index(['x2', 'x3', 'x8', 'x11', 'x14', 'x15'], dtype='object')),
                                ('imputador_c',
                                 SimpleImputer(strategy='most_frequent'),
                                 Index(['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10', 'x12', 'x13'], dtype='object'))])

In [203]:
X_train_Trans= transformador_columnas.transform(X_train)
X_test_Trans= transformador_columnas.transform(X_test)
colum_or=cols_n.tolist() + cols_c.tolist()
X_train_Trans=pd.DataFrame(X_train_Trans, columns=colum_or)
X_test_Trans=pd.DataFrame(X_test_Trans, columns=colum_or)  

In [204]:
X_01=X_train_Trans.append(X_test_Trans,ignore_index=True)
X_01

,x2,x3,x8,x11,x14,x15,x1,x4,x5,x6,x7,x9,x10,x12,x13
0,28,6.5,1.46,0.2,80,404.2,b,u,g,c,v,t,t,f,g
1,15.83,585,1.5,2,198.4,0,b,u,g,c,v,t,f,f,g
2,23.284,0,0,0,224,1,b,y,p,c,bb,t,f,t,s
3,38.92,76.792,0.5,0,300,2,b,u,g,c,v,t,f,t,g
4,33.5,6358.39,7,0,280,1510.6,b,y,g,c,v,t,f,t,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,22.966,258.458,26.2,0,162.2,33.8,b,u,g,c,v,t,f,f,g
496,31.45,11.5,176.55,0.4,20,16,b,y,g,c,h,f,f,f,g
497,28,442.35,0.75,0,103.2,6397.8,b,u,g,c,v,f,f,t,g
498,34.08,6.5,125,0,126,0,b,u,g,aa,v,t,f,f,g


In [205]:
model_dummy=("onehot_m",OneHotEncoder(sparse=False), ['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10', 'x12', 'x13'])
model_estandard=("estandarizador2",StandardScaler(),['x2', 'x3', 'x8', 'x11', 'x14', 'x15'])

trans_col1= ColumnTransformer(transformers=[model_dummy,model_estandard],remainder='passthrough')
trans_col1.fit(X_01)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot_m', OneHotEncoder(sparse=False),
                                 ['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10',
                                  'x12', 'x13']),
                                ('estandarizador2', StandardScaler(),
                                 ['x2', 'x3', 'x8', 'x11', 'x14', 'x15'])])

In [206]:
X_train_trans=trans_col1.transform(X_train_Trans)
X_test_trans=trans_col1.transform(X_test_Trans)

# Modelo Arbol

In [207]:
modelo_arbol = DecisionTreeClassifier(max_depth=2, random_state=123)       
modelo_arbol.fit(X_train_trans, Y_train)              
Y_pred = modelo_arbol.predict(X_test_trans)
ACT_final=accuracy_score(Y_test, Y_pred)
F1T_final=f1_score(Y_test,Y_pred, average='macro')

print("Evaluación del modelo Seleccionado: ")
print("Accuracy: ", round(ACT_final,3))
print("F1: ", round(F1T_final,3))

Evaluación del modelo Seleccionado: 
Accuracy:  0.72
F1:  0.714


# final

In [208]:
cols_nP=X.select_dtypes(np.number).columns  
cols_cP=X.select_dtypes(include=['object']).columns 

In [209]:
t1_modelP=('imputador_n', KNNImputer(n_neighbors= 5),cols_nP)
t2_modelP=('imputador_c',SimpleImputer(strategy='most_frequent'),cols_cP)

transformador_columnas=ColumnTransformer(transformers=[t1_modelP,t2_modelP],remainder='passthrough')
transformador_columnas.fit(X)

ColumnTransformer(remainder='passthrough',
                  transformers=[('imputador_n', KNNImputer(),
                                 Index(['x2', 'x3', 'x8', 'x11', 'x14', 'x15'], dtype='object')),
                                ('imputador_c',
                                 SimpleImputer(strategy='most_frequent'),
                                 Index(['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10', 'x12', 'x13'], dtype='object'))])

In [235]:
X_TransP1= transformador_columnas.transform(X)

colum_or=cols_n.tolist() + cols_c.tolist()
X_TransP1=pd.DataFrame(X_TransP1, columns=colum_or)

model_dummyP=("onehot_m",OneHotEncoder(sparse=False), ['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10', 'x12', 'x13'])
model_estandardP=("estandarizador2",StandardScaler(),['x2', 'x3', 'x8', 'x11', 'x14', 'x15'])

trans_col1P= ColumnTransformer(transformers=[model_dummyP,model_estandardP],remainder='passthrough')
trans_col1P.fit(X_TransP1)
 
X_TransP=trans_col1P.transform(X_TransP1)
X_TransP=pd.DataFrame(X_TransP) 

In [243]:
modelo_prod = DecisionTreeClassifier(max_depth=2, random_state=123) 
modelo_prod.fit(X_TransP, Y)

DecisionTreeClassifier(max_depth=2, random_state=123)

In [216]:
df2=pd.read_csv('data/japan_bank_190_X_conNulos_.csv')

In [238]:
cols_num=X.select_dtypes(np.number).columns  
cols_cat=X.select_dtypes(include=['object','bool']).columns  
t1df2=('imputador_num', KNNImputer(n_neighbors= 5),cols_num)
t2df2=('imputador_cat',SimpleImputer(strategy='most_frequent'),cols_cat)

In [239]:
transformador_columnas=ColumnTransformer(transformers=[t1df2,t2df2],remainder='passthrough')
transformador_columnas.fit(X)
df2_Transformado= transformador_columnas.transform(df2)

cols_orden=cols_num.tolist() + cols_cat.tolist()
df2_Transformado=pd.DataFrame(df2_Transformado, columns=cols_orden)


t_dummy=("onehot2",OneHotEncoder(sparse=False), ['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10', 'x12', 'x13'])
t_estandard=("estandarizador2",StandardScaler(),['x2', 'x3', 'x8', 'x11', 'x14', 'x15'])
transformador_columnas_tKnnE= ColumnTransformer(transformers=[t_dummy,t_estandard],remainder='passthrough')
transformador_columnas_tKnnE.fit(X_TransP1)

df2_t=transformador_columnas_tKnnE.transform(df2_Transformado)
df2_t=pd.DataFrame(df2_t)
df2_t

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,-0.116957,-0.398134,-0.325672,-0.022593,-0.216859,-0.355382
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.233789,-0.357907,-0.164035,0.038714,0.962695,-0.398283
2,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.986905,-0.370948,-0.157855,-0.635667,0.384954,-0.374568
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.641589,1.625555,-0.321515,-0.635667,1.618671,-0.331666
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,-0.202443,-0.396906,-0.293773,-0.635667,-1.179760,0.148866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,-0.680998,-0.270330,-0.322431,-0.635667,-0.036315,2.413232
186,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.280158,-0.398728,-0.082700,0.897017,0.096084,-0.091844
187,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.097807,-0.397860,4.329199,-0.635667,0.348845,-0.397839
188,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,-0.707189,-0.317292,-0.325178,3.042773,-0.457584,-0.146470


In [244]:
ypred=modelo_prod.predict(df2_t)

In [247]:
ypred= pd.DataFrame(ypred,columns=['ypred'])
ypred.to_csv('Cortez_Tomas.csv',index=False)